# Data Reading

In [0]:
df = spark.read.format('parquet')\
    .option('inferSchema',True)\
        .load('abfss://bronze@carkeerthiidatalake.dfs.core.windows.net/rawdata')

In [0]:
df.display()

# DATA TRANSFORMATION

In [0]:
from pyspark.sql.functions import *

In [0]:
df = df.withColumn('model_category',split(col('Model_ID'),'-')[0])

In [0]:
df.display()

In [0]:
df.withColumn('Units_Sold',col('Units_Sold').cast(StringType())).display()

In [0]:
df = df.withColumn('RevperUnit',col('Revenue')/col('Units_Sold'))
df.display()

# AD - HOC

In [0]:
df.groupby('Year','BranchName').agg(sum('units_sold').alias('Total_Units')).sort('Year','Total_Units',ascending =[1,0]).display()

In [0]:
display(df.groupby('Year','BranchName').agg(sum('units_sold').alias('Total_Units')).sort('Year','Total_Units',ascending =[1,0]))

Databricks visualization. Run in Databricks to view.

In [0]:
df.write.format('parquet')\
    .mode('overwrite')\
    .option('path', 'abfss://silver@carkeerthiidatalake.dfs.core.windows.net/carsales')\
    .save()

# Querying Silver Data

In [0]:
%sql
select * from parquet.`abfss://silver@carkeerthiidatalake.dfs.core.windows.net/carsales`